In [1]:
# importing package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
import pickle
from datetime import date
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
from pathlib import Path
from random import sample 


#from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm 


os.chdir('/Volumes/Hh/') ## plug in the small blue usb called Hb
DATA_STORE = Path('/Volumes/Hh/data/assets.h5') ## plug the small blue usb called Hb
idx = pd.IndexSlice

In [2]:
# pull stock tickers for daily scan
with pd.HDFStore(DATA_STORE) as store:
    print(store.keys())
    stocks = store['us_equities/stocks']

tickers=[t.strip().replace('/','-') for t in list(stocks.index)]
len(tickers)

['/engineered_features', '/engineered_features_dummied', '/engineered_features_dummied_v2', '/us_equities/stocks', '/sp500/fred', '/quandl/wiki/prices', '/quandl/wiki/stocks', '/fred/assets']


3239

In [3]:
# download data and stack to long format(date, tickert * close price)
start_date=(pd.Timestamp(date.today().strftime('%Y-%m-%d'))-pd.Timedelta(days=30*24)).strftime('%Y-%m-%d')
print('pulling data starting from:',start_date)

data = yf.download(tickers=tickers, start=start_date, interval="1d", threads=False).sort_index()

print("max vs min of date and # records:",np.max(data.index),np.min(data.index))
print('num of tickers:', len(data['Adj Close'].columns))
data.tail()

pulling data starting from: 2022-11-21


[*********************100%%**********************]  3239 of 3239 completed

56 Failed downloads:
['RILYO', 'SIX', 'SPWR', 'GPS', 'MORF', 'OXLCM', 'ETRN', 'PXD', 'CPG', 'DRQ', 'SP', 'AQNU', 'PNM', 'SNPO', 'SWAV', 'OLK', 'ALPN', 'AGTI', 'DCPH', 'NWLI', 'NSS', 'EVBG', 'PWFL', 'SLCA', 'ATRI', 'MCBC', 'NEWTL', 'UGIC', 'AIRC', 'DO', 'CPLP', 'ERF', 'CERE', 'AEL', 'SMMF', 'HRT', 'FUSN', 'HIBB', 'WRK', 'OSG', 'DOOR', 'UCBIO', 'TNP', 'SBOW', 'AMK', 'TGAN', 'NS', 'CNHI', 'UCBI', 'TAST', 'KAMN', 'MDC', 'MODN', 'MOR', 'WIRE']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SITC']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-11-21 -> 2024-11-10)')


max vs min of date and # records: 2024-11-08 00:00:00 2022-11-21 00:00:00
num of tickers: 3239


Adj Close                                                      \
                     A         AA    AAL       AAOI        AAON        AAP   
Date                                                                         
2024-11-04  139.649994  40.730000  12.99  16.400000  112.459999  37.450001   
2024-11-05  140.410004  42.779999  13.19  16.540001  113.019997  38.160000   
2024-11-06  137.779999  45.360001  14.00  16.209999  120.250000  38.840000   
2024-11-07  139.630005  45.689999  13.62  17.900000  119.099998  38.980000   
2024-11-08  136.679993  44.779999  13.81  27.760000  138.240005  37.900002   

                                                          ... Volume          \
                  AAPL        AAT         AB        ABBV  ...  ZIONP     ZIP   
Date                                                      ...                  
2024-11-04  221.766006  27.129999  36.110001  200.470001  ...   4895  221600   
2024-11-05  223.204422  27.320000  36.490002  201.679993  ...   3865  295000   
2024-11-06  222.475235  28.020000  38.209999  201.199997  ...   7864  668700   
2024-11-07  227.229996  27.959999  37.549999  200.509995  ...  12328  995800   
2024-11-08  226.960007  27.940001  36.709999  199.500000  ...   9431  541300   

                                                                           \
                 ZM     ZNTL       ZS      ZTS      ZUO      ZWS     ZYME   
Date                                                                        
2024-11-04  1551100  2853400   832900  4164000  2744800   840300  1034500   
2024-11-05  2568400  1293500   992100  2453800  2953500  1309000   365700   
2024-11-06  3880300  1332600  1745600  5362100  2670700  1971700   605800   
2024-11-07  2288300  1330100  1876000  3781300  2968400  1612600   888900   
2024-11-08  2838800  1461400  1061600  3243400  3536300  1058900   678700   

                    
              ZYXI  
Date                
2024-11-04  110700  
2024-11-05   86800  
2024-11-06  244000  
2024-11-07   93100  
2024-11-08   84000  

[5 rows x 19434 columns]

In [4]:
# calculate latest trading date and original anchor date for sampleing 

latest_date=data.index.tolist()[-1]
print("latest trading data date:",latest_date.strftime('%Y-%m-%d'))

origin_date=(latest_date-pd.Timedelta(days=30*36)).strftime('%Y-%m-%d')
print("origin date:",origin_date)

# resample every 30 days
data_resampled=data.resample('30D',origin=origin_date).first()
data_resampled.tail()

latest trading data date: 2024-11-08
origin date: 2021-11-24


Adj Close                                                  \
                     A         AA    AAL   AAOI        AAON        AAP   
Date                                                                     
2024-07-11  130.820007  38.957489  10.73   9.23   88.489014  61.069038   
2024-08-10  135.600006  31.114681   9.61   7.85   87.310089  57.671902   
2024-09-09  138.089996  28.900040  11.22  13.02   88.589996  38.461178   
2024-10-09  144.800003  38.347172  11.93  16.17  109.480003  38.183048   
2024-11-08  136.679993  44.779999  13.81  27.76  138.240005  37.900002   

                                                          ... Volume          \
                  AAPL        AAT         AB        ABBV  ...  ZIONP     ZIP   
Date                                                      ...                  
2024-07-11  227.057098  22.899040  32.579651  167.029556  ...   3201  426200   
2024-08-10  217.290939  24.143230  32.605167  188.673325  ...   2016  640300   
2024-09-09  220.667221  25.510000  32.654140  194.863907  ...   5464  540200   
2024-10-09  229.287735  26.389999  34.015545  193.207138  ...    446  285400   
2024-11-08  226.960007  27.940001  36.709999  199.500000  ...   9431  541300   

                                                                          \
                 ZM     ZNTL       ZS      ZTS      ZUO      ZWS    ZYME   
Date                                                                       
2024-07-11  3515300  1279700  1472400  1989900   994200  1339200  735000   
2024-08-10  2357200  1859000   787500  1638100  1020500   576800  660500   
2024-09-09  2780700   965500  2381400  2652700  1173400  6101900  796600   
2024-10-09  4297200  1229100  2669600   965400  1321000   878600  305900   
2024-11-08  2838800  1461400  1061600  3243400  3536300  1058900  678700   

                    
              ZYXI  
Date                
2024-07-11  101700  
2024-08-10  100200  
2024-09-09   86400  
2024-10-09   41900  
2024-11-08   84000  

[5 rows x 19434 columns]

In [5]:
# generate metrics for monthly return: retutrn_1m = current_price/last_mthend_price - 1
outlier_cutoff = 0.01
data = pd.DataFrame()
lags = [1, 2, 3, 6, 9, 12]
for lag in lags:
    data[f'return_{lag}m'] = (data_resampled['Adj Close']
                           .pct_change(lag)
                           .stack()
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
data = data.swaplevel().dropna()
data.index.set_names(["ticker", "date"], inplace=True)

print('dp of each ticker:', data.groupby(level=['ticker']).size()[0])
data.info()
data.head()


dp of each ticker: 13
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 40512 entries, ('A', Timestamp('2023-11-14 00:00:00')) to ('ZYXI', Timestamp('2024-11-08 00:00:00'))
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   return_1m   40512 non-null  float64
 1   return_2m   40512 non-null  float64
 2   return_3m   40512 non-null  float64
 3   return_6m   40512 non-null  float64
 4   return_9m   40512 non-null  float64
 5   return_12m  40512 non-null  float64
dtypes: float64(6)
memory usage: 2.1+ MB


,,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m
ticker,date,,,,,,
A,2023-11-14,0.000987,-0.017735,-0.026878,-0.023383,-0.030461,-0.021079
AA,2023-11-14,-0.019702,-0.045547,-0.035698,-0.054137,-0.061510,-0.046054
AAL,2023-11-14,0.025963,-0.040646,-0.074245,-0.033727,-0.031635,-0.010178
AAOI,2023-11-14,0.452071,0.117125,-0.019567,0.163136,0.127959,0.109216
AAON,2023-11-14,0.111574,0.025785,0.005740,0.008608,0.021000,0.016053


In [6]:
# rolling french factor beta calc
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=origin_date)[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'date'
factor_data.info()
factor_data.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35 entries, 2021-11-30 to 2024-09-30
Freq: ME
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  35 non-null     float64
 1   SMB     35 non-null     float64
 2   HML     35 non-null     float64
 3   RMW     35 non-null     float64
 4   CMA     35 non-null     float64
dtypes: float64(5)
memory usage: 1.6 KB


,Mkt-RF,SMB,HML,RMW,CMA
date,,,,,
2024-05-31,0.0434,0.0076,-0.0166,0.0298,-0.0307
2024-06-30,0.0277,-0.0437,-0.0331,0.0051,-0.0178
2024-07-31,0.0124,0.0828,0.0573,0.0022,0.0043
2024-08-31,0.0161,-0.0365,-0.0113,0.0085,0.0086
2024-09-30,0.0174,-0.0103,-0.0260,0.0003,-0.0026


In [7]:
# join return_1m
data_return_1m=data.reset_index(level='date').astype('str')
data_return_1m['date']=pd.to_datetime(data_return_1m['date']).dt.strftime('%Y-%m')
data_return_1m=data_return_1m[['date','return_1m']]

factor_data_tomerge=factor_data.reset_index()
factor_data_tomerge['date']=pd.to_datetime(factor_data_tomerge['date']).dt.strftime('%Y-%m')

return_factor_joined = data_return_1m.reset_index().merge(factor_data_tomerge, how='inner', on='date')
return_factor_joined.dropna(inplace=True)
return_factor_joined.info()
return_factor_joined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34215 entries, 0 to 34214
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     34215 non-null  object 
 1   date       34215 non-null  object 
 2   return_1m  34215 non-null  object 
 3   Mkt-RF     34215 non-null  float64
 4   SMB        34215 non-null  float64
 5   HML        34215 non-null  float64
 6   RMW        34215 non-null  float64
 7   CMA        34215 non-null  float64
dtypes: float64(5), object(3)
memory usage: 2.1+ MB


,ticker,date,return_1m,Mkt-RF,SMB,HML,RMW,CMA
0,A,2023-11,0.000986573887636899,0.0883,-0.0011,0.0166,-0.0381,-0.0099
1,AA,2023-11,-0.019701920759437996,0.0883,-0.0011,0.0166,-0.0381,-0.0099
2,AAL,2023-11,0.02596318513494844,0.0883,-0.0011,0.0166,-0.0381,-0.0099
3,AAOI,2023-11,0.4520707105856354,0.0883,-0.0011,0.0166,-0.0381,-0.0099
4,AAON,2023-11,0.1115739005292733,0.0883,-0.0011,0.0166,-0.0381,-0.0099


In [8]:
# regression to calc french fama beta
def regress(data, yvar, xvars):
    Y = data[yvar].astype('float')
    X = data[xvars]
    result = sm.OLS(Y, X).fit()
    return result.params

beta=return_factor_joined.groupby('ticker').apply(regress, 'return_1m', ['Mkt-RF','SMB','HML','RMW','CMA'])


data=data.join(beta)
data.head()

,,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m,Mkt-RF,SMB,HML,RMW,CMA
ticker,date,,,,,,,,,,,
A,2023-11-14,0.000987,-0.017735,-0.026878,-0.023383,-0.030461,-0.021079,1.912806,1.141697,-0.989961,1.686154,5.061331
AA,2023-11-14,-0.019702,-0.045547,-0.035698,-0.054137,-0.061510,-0.046054,-1.236647,-0.235729,3.367765,-0.395041,-7.098131
AAL,2023-11-14,0.025963,-0.040646,-0.074245,-0.033727,-0.031635,-0.010178,1.275653,3.314910,-3.864741,0.183899,3.071927
AAOI,2023-11-14,0.452071,0.117125,-0.019567,0.163136,0.127959,0.109216,2.913493,4.391521,-7.556747,-7.566218,6.733216
AAON,2023-11-14,0.111574,0.025785,0.005740,0.008608,0.021000,0.016053,-0.663462,0.129295,1.695823,-3.616042,-3.077187


In [9]:
# lag 1m returns
for t in range(1, 7):
    data[f'return_1m_t-{t}'] = data.groupby(level='ticker').return_1m.shift(t)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 40512 entries, ('A', Timestamp('2023-11-14 00:00:00')) to ('ZYXI', Timestamp('2024-11-08 00:00:00'))
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   return_1m      40512 non-null  float64
 1   return_2m      40512 non-null  float64
 2   return_3m      40512 non-null  float64
 3   return_6m      40512 non-null  float64
 4   return_9m      40512 non-null  float64
 5   return_12m     40512 non-null  float64
 6   Mkt-RF         40495 non-null  float64
 7   SMB            40495 non-null  float64
 8   HML            40495 non-null  float64
 9   RMW            40495 non-null  float64
 10  CMA            40495 non-null  float64
 11  return_1m_t-1  37361 non-null  float64
 12  return_1m_t-2  34215 non-null  float64
 13  return_1m_t-3  31075 non-null  float64
 14  return_1m_t-4  27943 non-null  float64
 15  return_1m_t-5  24820 non-null  float64
 16  return_1m_t-6  2170

,,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m,Mkt-RF,SMB,HML,RMW,CMA,return_1m_t-1,return_1m_t-2,return_1m_t-3,return_1m_t-4,return_1m_t-5,return_1m_t-6
ticker,date,,,,,,,,,,,,,,,,,
A,2023-11-14,0.000987,-0.017735,-0.026878,-0.023383,-0.030461,-0.021079,1.912806,1.141697,-0.989961,1.686154,5.061331,NaN,NaN,NaN,NaN,NaN,NaN
AA,2023-11-14,-0.019702,-0.045547,-0.035698,-0.054137,-0.061510,-0.046054,-1.236647,-0.235729,3.367765,-0.395041,-7.098131,NaN,NaN,NaN,NaN,NaN,NaN
AAL,2023-11-14,0.025963,-0.040646,-0.074245,-0.033727,-0.031635,-0.010178,1.275653,3.314910,-3.864741,0.183899,3.071927,NaN,NaN,NaN,NaN,NaN,NaN
AAOI,2023-11-14,0.452071,0.117125,-0.019567,0.163136,0.127959,0.109216,2.913493,4.391521,-7.556747,-7.566218,6.733216,NaN,NaN,NaN,NaN,NaN,NaN
AAON,2023-11-14,0.111574,0.025785,0.005740,0.008608,0.021000,0.016053,-0.663462,0.129295,1.695823,-3.616042,-3.077187,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# momentum features
for lag in [2,3,6,9,12]:
    data[f'momentum_{lag}'] = data[f'return_{lag}m'].sub(data.return_1m)
data[f'momentum_3_12'] = data[f'return_12m'].sub(data.return_3m)

dates = data.index.get_level_values('date')
data['month'] = dates.month

data.tail()

,,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m,Mkt-RF,SMB,HML,RMW,...,return_1m_t-4,return_1m_t-5,return_1m_t-6,momentum_2,momentum_3,momentum_6,momentum_9,momentum_12,momentum_3_12,month
ticker,date,,,,,,,,,,,,,,,,,,,,,
ZTS,2024-11-08,-0.067997,-0.038532,-0.012061,0.009452,-0.010929,0.002814,1.976523,2.507431,-3.809600,1.770004,...,-0.004481,0.063308,0.122741,0.029465,0.055936,0.077449,0.057068,0.070811,0.014875,11
ZUO,2024-11-08,0.121591,0.073796,0.053142,-0.004978,0.013344,0.014542,1.082608,0.788314,-0.858710,1.428737,...,-0.105473,-0.011799,0.182558,-0.047795,-0.068449,-0.126569,-0.108247,-0.107049,-0.038600,11
ZWS,2024-11-08,0.079063,0.106374,0.093790,0.032203,0.022409,0.023490,0.441642,0.321240,-0.702242,-1.347822,...,-0.011206,-0.065585,0.020056,0.027310,0.014726,-0.046860,-0.056654,-0.055573,-0.070299,11
ZYME,2024-11-08,0.306363,0.192211,0.170876,0.102901,0.038664,0.067081,2.095855,1.187753,-3.039409,-0.734094,...,0.027055,0.040043,-0.001081,-0.114152,-0.135487,-0.203461,-0.267699,-0.239282,-0.103795,11
ZYXI,2024-11-08,0.168612,0.072708,0.068783,-0.027942,-0.036831,-0.002188,-0.562845,0.548690,0.257372,-5.190987,...,0.011879,-0.132959,-0.103275,-0.095904,-0.099829,-0.196554,-0.205443,-0.170801,-0.070971,11


In [11]:
# append sector info
data = data.join(stocks['Sector'])
data.rename(columns={'Sector':'sector'},inplace=True)
data.sector = data.sector.fillna('Unknown')

In [12]:
#dummy data
dummy_data = pd.get_dummies(data,
                            columns=['month', 'sector'],
                            prefix=['month', ''],
                            prefix_sep=['_', ''])
dummy_data = dummy_data.rename(columns={c:c.replace('.0', '') for c in dummy_data.columns})
dummy_data=dummy_data.loc[idx[:,latest_date],:].dropna()
dummy_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3112 entries, ('A', Timestamp('2024-11-08 00:00:00')) to ('ZYXI', Timestamp('2024-11-08 00:00:00'))
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   return_1m               3112 non-null   float64
 1   return_2m               3112 non-null   float64
 2   return_3m               3112 non-null   float64
 3   return_6m               3112 non-null   float64
 4   return_9m               3112 non-null   float64
 5   return_12m              3112 non-null   float64
 6   Mkt-RF                  3112 non-null   float64
 7   SMB                     3112 non-null   float64
 8   HML                     3112 non-null   float64
 9   RMW                     3112 non-null   float64
 10  CMA                     3112 non-null   float64
 11  return_1m_t-1           3112 non-null   float64
 12  return_1m_t-2           3112 non-null   float64
 13  return_1m_t-3    

In [13]:
# import model and vars
with open(os.path.join(os.getcwd(), 'data/model_var/variables.pkl'),'rb') as file:
    regressor,X_test=pickle.load(file)


In [14]:
# complete final dataset by making sure the same column layout with model
missing_col=[i for i in X_test.columns if i not in dummy_data.columns]
for m in missing_col:
    dummy_data[m]=False

In [15]:
# train and print prediction outcome
y_pred_today = regressor.predict(dummy_data[X_test.columns])
output_file=os.path.join(os.getcwd(),'data/daily_scan_output',latest_date.strftime('%Y_%m_%d')+'.xlsx')
pd.DataFrame({'return_1m':dummy_data.return_1m,'y_pred':y_pred_today}).sort_values(by='y_pred', ascending=False).to_excel(output_file)